In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

Reusing TensorBoard on port 6007 (pid 20335), started 0:08:13 ago. (Use '!kill 20335' to kill it.)

In [3]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [4]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [5]:
SIZE = 64
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(), 
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE),
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        return img_gray, img_ab
    
    def __len__(self):
        return len(self.paths)

In [6]:
# Training
batch_size = 128
train_imagefolder = LabImageFolder('../../datasets/dataset/train/colour')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder('../../datasets/dataset/test/colour' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [7]:
kernel_size=3
stride_en=1
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'


class ResBlock(nn.Module):
    def __init__(self, in_size, out_size, kernel_size, stride, padding, padding_mode, encoder):
        super().__init__()
        if (encoder):
            self.conv1 = nn.Conv2d(in_size, in_size, kernel_size=kernel_size, stride=stride, padding=padding, padding_mode=padding_mode)
            self.conv2 = nn.Conv2d(in_size, out_size, kernel_size=kernel_size, stride=stride, padding=padding, padding_mode=padding_mode)
        else:
            self.conv1 = nn.ConvTranspose2d(in_size, in_size, kernel_size=kernel_size, stride=stride, padding=padding, padding_mode=padding_mode)
            self.conv2 = nn.ConvTranspose2d(in_size, out_size, kernel_size=kernel_size, stride=stride, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(in_size)
        self.batchnorm2 = nn.BatchNorm2d(out_size)

    def convblock(self, x):
        x = F.leaky_relu(self.batchnorm1(self.conv1(x)), negative_slope=0.1)
        x = F.leaky_relu(self.batchnorm1(self.conv1(x)), negative_slope=0.1)
        x = F.leaky_relu(self.batchnorm1(self.conv1(x)), negative_slope=0.1)
        x = F.leaky_relu(self.batchnorm2(self.conv2(x)), negative_slope=0.1)
        return x
   
    def forward(self, x): 
        y = F.leaky_relu(self.batchnorm2(self.conv2(x)), negative_slope=0.1)
        return y + self.convblock(x)
    
    
    
class ColorizationNet(nn.Module):
    def __init__(self, input_size=128):
        super(ColorizationNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 16, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        self.batchnorm1 = nn.BatchNorm2d(16)
        self.resblock1 = ResBlock(in_size=16, out_size=32, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode, encoder=True)
        self.resblock2 = ResBlock(in_size=32, out_size=64, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode, encoder=True)
        
        self.convtrans1 = nn.ConvTranspose2d(64, 32, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.resblock3 = ResBlock(in_size=32, out_size=16, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode, encoder=False)
        self.resblock4 = ResBlock(in_size=16, out_size=8, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode, encoder=False) 
        self.convtrans2 = nn.ConvTranspose2d(8, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)

    def forward(self, input):
        # encoder
        x = F.leaky_relu(self.batchnorm1(self.conv1(input)), negative_slope=0.1)
        x = F.avg_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = F.avg_pool2d(self.resblock1(x), kernel_size=3, stride=2, padding=1)
        x = F.avg_pool2d(self.resblock2(x), kernel_size=3, stride=2, padding=1)
        # decoder
        x = F.leaky_relu(self.batchnorm2(self.convtrans1(x)), negative_slope=0.1)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = F.interpolate(self.resblock3(x), scale_factor=scale_factor)
        x = F.interpolate(self.convtrans2(self.resblock4(x)), scale_factor=scale_factor)
        return x

In [8]:
model = ColorizationNet()

In [9]:
criterion = nn.MSELoss()

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [11]:
# # Move model and loss function to GPU
if use_gpu: 
    criterion = criterion.cuda()
    model = model.cuda()

In [12]:
if use_gpu: 
    from torchsummary import summary

    summary(model, (1, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 64, 64]             160
       BatchNorm2d-2           [-1, 16, 64, 64]              32
            Conv2d-3           [-1, 32, 32, 32]           4,640
       BatchNorm2d-4           [-1, 32, 32, 32]              64
            Conv2d-5           [-1, 16, 32, 32]           2,320
       BatchNorm2d-6           [-1, 16, 32, 32]              32
            Conv2d-7           [-1, 16, 32, 32]           2,320
       BatchNorm2d-8           [-1, 16, 32, 32]              32
            Conv2d-9           [-1, 16, 32, 32]           2,320
      BatchNorm2d-10           [-1, 16, 32, 32]              32
           Conv2d-11           [-1, 32, 32, 32]           4,640
      BatchNorm2d-12           [-1, 32, 32, 32]              64
         ResBlock-13           [-1, 32, 32, 32]               0
           Conv2d-14           [-1, 64,

In [13]:
class AverageMeter(object):
  '''A handy class from the PyTorch ImageNet tutorial''' 
  def __init__(self):
    self.reset()
  def reset(self):
    self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
  def update(self, val, n=1):
    self.val = val
    self.sum += val * n
    self.count += n
    self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
  '''Show/save rgb image from grayscale and ab channels
     Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
  plt.clf() # clear matplotlib 
  color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
  color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
  color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
  color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
  color_image = lab2rgb(color_image.astype(np.float64))
  grayscale_input = grayscale_input.squeeze().numpy()
  if save_path is not None and save_name is not None: 
    plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
    plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [14]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'

In [15]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = AverageMeter()

    model.eval()
    already_saved_images = False
    for gray, ab in val_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()

        # Run model and record loss
        output_ab = model(gray) # throw away class predictions
        loss = criterion(output_ab, ab)
        
        _loss.update(loss.item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print('Validate: Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
         loss=_loss))

    print('Finished validation.')
    if epoch >= 0:
        writer.add_scalar("Loss/test", _loss.avg, epoch)
    return _loss.avg

In [16]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = AverageMeter()
    
    model.train()

    for gray, ab in train_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray) 
        loss = criterion(output_ab, ab) 
        loss.backward()
        optimizer.step()
        
        _loss.update(loss.item(), gray.size(0))
        
    print(f'Epoch: {epoch}, Loss {_loss.val:.4f} ({_loss.avg:.4f})')
    
    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("Loss/train", _loss.avg, epoch)

In [17]:
# Make folders and set parameters
checkpoints = 'checkpoints'
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = 1e10
epochs = 1000

In [18]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses < best_losses:
        best_losses = losses
        torch.save(model.state_dict(), '{}/model-epoch-{}-losses-{:.6f}.pth'.format(checkpoints,epoch+1,losses))

    if epoch == epochs - 1:
        torch.save(model.state_dict(), '{}/model-epoch-{}-last-losses-{:.6f}.pth'.format(checkpoints,epoch+1,losses))


Starting training epoch 0
Epoch: 0, Loss 0.0111 (0.0967)
Finished training epoch 0


/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 910 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 1478 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 1731 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 296 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 1102 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 1441 pixels
  return func(*args

Validate: Loss 0.0589 (0.0559)	
Finished validation.
Starting training epoch 1
Epoch: 1, Loss 0.0082 (0.0102)
Finished training epoch 1


/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 1052 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 1550 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 2077 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 307 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 1146 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 1755 pixels
  return func(*arg

Validate: Loss 0.0485 (0.0454)	
Finished validation.
Starting training epoch 2
Epoch: 2, Loss 0.0087 (0.0088)
Finished training epoch 2


...

Validate: Loss 0.0071 (0.0069)	
Finished validation.
Starting training epoch 638
Epoch: 638, Loss 0.0032 (0.0031)
Finished training epoch 638
Validate: Loss 0.0068 (0.0069)	
Finished validation.
Starting training epoch 639
Epoch: 639, Loss 0.0030 (0.0031)
Finished training epoch 639
Validate: Loss 0.0069 (0.0068)	
Finished validation.
Starting training epoch 640
Epoch: 640, Loss 0.0031 (0.0030)
Finished training epoch 640
Validate: Loss 0.0068 (0.0069)	
Finished validation.
Starting training epoch 641
Epoch: 641, Loss 0.0030 (0.0031)
Finished training epoch 641
Validate: Loss 0.0068 (0.0069)	
Finished validation.
Starting training epoch 642
Epoch: 642, Loss 0.0032 (0.0031)
Finished training epoch 642
Validate: Loss 0.0071 (0.0071)	
Finished validation.
Starting training epoch 643
Epoch: 643, Loss 0.0029 (0.0030)
Finished training epoch 643
Validate: Loss 0.0068 (0.0069)	
Finished validation.
Starting training epoch 644
Epoch: 644, Loss 0.0030 (0.0031)
Finished training epoch 644
Valida

/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 34 pixels
  return func(*args, **kwargs)


Validate: Loss 0.0075 (0.0073)	
Finished validation.
Starting training epoch 674
Epoch: 674, Loss 0.0029 (0.0030)
Finished training epoch 674
Validate: Loss 0.0070 (0.0069)	
Finished validation.
Starting training epoch 675
Epoch: 675, Loss 0.0032 (0.0030)
Finished training epoch 675
Validate: Loss 0.0070 (0.0069)	
Finished validation.
Starting training epoch 676
Epoch: 676, Loss 0.0028 (0.0030)
Finished training epoch 676
Validate: Loss 0.0068 (0.0068)	
Finished validation.
Starting training epoch 677
Epoch: 677, Loss 0.0033 (0.0030)
Finished training epoch 677
Validate: Loss 0.0073 (0.0073)	
Finished validation.
Starting training epoch 678
Epoch: 678, Loss 0.0030 (0.0031)
Finished training epoch 678
Validate: Loss 0.0074 (0.0072)	
Finished validation.
Starting training epoch 679
Epoch: 679, Loss 0.0033 (0.0030)
Finished training epoch 679
Validate: Loss 0.0069 (0.0070)	
Finished validation.
Starting training epoch 680
Epoch: 680, Loss 0.0032 (0.0031)
Finished training epoch 680
Valida

KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>

In [ ]:
torch.save(model.state_dict(), '{}/model-epoch-last-losses-{:.6f}.pth'.format(checkpoints,losses))

In [ ]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

In [ ]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()